In [ ]:
from ptlpinns.odes import equations, numerical
from ptlpinns.models import model, transfer
import numpy as np
import time
import torch
import json

In [ ]:
N_ITER = 600

zeta_list = [0.1, 0.5, 5, 10, 40, 60, 80]
N = 5120
t_span = (0, 10)
ic = np.array([1, 0]) 
t_eval = np.linspace(t_span[0], t_span[1], N)

RK45_time, Radau_time, PTL_PINN_time = [], [], []

### RK45 and Radau times

In [ ]:
for zeta in zeta_list:

    ode = equations.ode_oscillator_1D(w_0=1, zeta=zeta, forcing_1D=[lambda t: np.cos(t)])

    start_RK45 = time.time()
    numerical.solve_ode_equation(ode, t_span, t_eval, ic, method="RK45", rtol=1e-5, atol=1e-5)[0]
    end_RK45 = time.time()
    RK45_time.append(end_RK45 - start_RK45)

    start_Radau = time.time()
    numerical.solve_ode_equation(ode, t_span, t_eval, ic, method="Radau", rtol=1e-5, atol=1e-5)[0]
    end_Radau = time.time()
    Radau_time.append(end_Radau - start_Radau)

### PTL-PINNs

In [ ]:
undamped_path = "/home/dda24/PTL-PINNs/ptlpinns/models/train/undamped_k12"
undamped_name = "model_undamped_k12.pth"
undamped_model, _ = model.load_model(undamped_path, undamped_name)

underdamped_path = "/home/dda24/PTL-PINNs/ptlpinns/models/train/underdamped_k12"
underdamped_name = "model_underdamped_k12.pth"
underdamped_model, _ = model.load_model(underdamped_path, underdamped_name)

overdamped_path = "/home/dda24/PTL-PINNs/ptlpinns/models/train/overdamped_k12"
overdamped_name = "model_overdamped_k12.pth"
overdamped_model, _ = model.load_model(underdamped_path, underdamped_name)

In [ ]:
# Compute latent representation: H(t) and derivatives
# compute_H_dict(model, N=N, bias=True, t_span=(t_span[0], t_span[1]))
# compute_H_dict(model, N=N, bias=True, t_span=(t_span[0], t_span[1]))
# compute_H_dict(model, N=N, bias=True, t_span=(t_span[0], t_span[1]))

transfer.compute_H_dict(H_dict, w_0, zeta)

def compute_M(H_dict, w_0, zeta):
    N = H_dict['N']
    H_ic_0 = H_dict['H_ic']
    M = (H_dict['H_star'].T @ H_dict['H_star']) / N + (H_ic_0.T @ H_ic_0)  # shape (W, W)
    Minv = np.linalg.pinv(M)  # shape (W, W)
    return M, Minv

#
#
#

In [ ]:
# for zeta in zeta_list:

### Comparison

In [4]:
for i in range(len(zeta_list)):
    print(f"zeta: {zeta_list[i]} | RK45: {RK45_time[i]} | Radau: {Radau_time[i]} | PTL-PINN: {PTL_PINN_time[i]}")

IndexError: list index out of range